In [43]:
from torchvision import models

In [44]:
dir(models)

['AlexNet',
 'AlexNet_Weights',
 'ConvNeXt',
 'ConvNeXt_Base_Weights',
 'ConvNeXt_Large_Weights',
 'ConvNeXt_Small_Weights',
 'ConvNeXt_Tiny_Weights',
 'DenseNet',
 'DenseNet121_Weights',
 'DenseNet161_Weights',
 'DenseNet169_Weights',
 'DenseNet201_Weights',
 'EfficientNet',
 'EfficientNet_B0_Weights',
 'EfficientNet_B1_Weights',
 'EfficientNet_B2_Weights',
 'EfficientNet_B3_Weights',
 'EfficientNet_B4_Weights',
 'EfficientNet_B5_Weights',
 'EfficientNet_B6_Weights',
 'EfficientNet_B7_Weights',
 'EfficientNet_V2_L_Weights',
 'EfficientNet_V2_M_Weights',
 'EfficientNet_V2_S_Weights',
 'GoogLeNet',
 'GoogLeNetOutputs',
 'GoogLeNet_Weights',
 'Inception3',
 'InceptionOutputs',
 'Inception_V3_Weights',
 'MNASNet',
 'MNASNet0_5_Weights',
 'MNASNet0_75_Weights',
 'MNASNet1_0_Weights',
 'MNASNet1_3_Weights',
 'MaxVit',
 'MaxVit_T_Weights',
 'MobileNetV2',
 'MobileNetV3',
 'MobileNet_V2_Weights',
 'MobileNet_V3_Large_Weights',
 'MobileNet_V3_Small_Weights',
 'RegNet',
 'RegNet_X_16GF_Weights'

In [45]:
# alexnet = models.AlexNet() # not initialized
resnet = models.resnet101(pretrained=True)

In [46]:
resnet

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [47]:
from torchvision import transforms
preprocess = transforms.Compose([transforms.Resize(256),
                                 transforms.CenterCrop(224),
                                 transforms.ToTensor(),
                                 # mean 和 std 为什么要这样设置？因为这是ImageNet百万张图片的均值和标准差
                                 transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                                      std=[0.229, 0.224, 0.225])])

transforms.Normalize()涉及的知识：归一化和标准化。

参考链接：https://zhuanlan.zhihu.com/p/29957294

标准化后可以更加容易地得出最优参数 w 和 b 以及计算出 J(w,b)的最小值，从而达到加速收敛的效果。

In [48]:
from PIL import Image
img = Image.open('bobby.png').convert('RGB')

In [49]:
img.show()

In [50]:
img_t = preprocess(img)

In [51]:
img_t

tensor([[[-1.1932, -1.1932, -1.1932,  ..., -0.7479, -0.7650, -0.7479],
         [-1.2103, -1.2103, -1.2103,  ..., -0.7308, -0.7479, -0.7308],
         [-1.2103, -1.2103, -1.2103,  ..., -0.7137, -0.7137, -0.6794],
         ...,
         [ 1.1358,  1.1187,  1.1187,  ...,  0.4166,  0.3823,  0.3481],
         [ 1.1358,  1.1015,  1.1015,  ...,  0.4166,  0.3823,  0.3309],
         [ 1.1015,  1.1015,  1.0844,  ...,  0.4166,  0.3823,  0.3652]],

        [[-1.3880, -1.3880, -1.3704,  ..., -1.0728, -1.0728, -1.0553],
         [-1.3704, -1.3704, -1.3704,  ..., -1.0728, -1.0903, -1.0728],
         [-1.3529, -1.3529, -1.3704,  ..., -1.0553, -1.0903, -1.0728],
         ...,
         [ 0.5028,  0.4853,  0.4678,  ..., -0.1800, -0.1625, -0.1800],
         [ 0.4853,  0.4678,  0.4503,  ..., -0.1625, -0.1625, -0.1975],
         [ 0.4678,  0.4678,  0.4503,  ..., -0.2150, -0.2500, -0.2325]],

        [[-1.2990, -1.2990, -1.2990,  ..., -1.1247, -1.1596, -1.1596],
         [-1.2990, -1.2990, -1.2990,  ..., -1

In [52]:
import torch
batch_t = torch.unsqueeze(img_t, 0) # 总而言之就是添加 batch channel

In [53]:
resnet.eval() # 简而言之，就是评估模式。而非训练模式。这个模式下，dropout和batch normalization都会被关闭。

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [54]:
out = resnet(batch_t)

In [55]:
out

tensor([[-3.4334e+00, -1.5614e+00, -2.4276e+00, -3.2404e+00, -3.2074e+00,
         -1.2164e+00, -1.6795e+00, -2.8699e+00, -1.6355e+00, -2.6301e+00,
         -1.8179e+00, -1.3811e+00, -2.3570e+00, -3.3602e+00, -2.7653e+00,
         -2.9508e+00, -3.5321e+00, -1.1526e+00, -1.0569e+00, -7.4045e-01,
         -2.8844e+00, -4.2528e+00, -2.2540e+00, -1.0095e+00, -1.1785e+00,
         -7.0184e-01, -2.8587e+00, -2.4700e+00, -1.9621e+00, -2.9967e+00,
         -3.2203e+00, -1.4650e+00, -1.4419e+00, -2.1321e+00, -1.6798e+00,
         -3.1890e+00, -1.2127e+00, -1.3585e+00, -1.0065e+00, -1.2804e+00,
         -7.6743e-01, -1.3957e+00,  1.0368e+00, -8.8011e-02, -2.0104e+00,
         -1.1758e+00,  4.7391e-01, -1.4301e+00, -2.8250e+00, -2.8126e+00,
         -2.2023e+00, -1.4232e+00, -1.7612e+00, -1.7860e+00, -1.6881e+00,
         -1.2825e+00, -9.4695e-02, -1.8799e+00, -3.0443e+00, -5.1067e-01,
          1.5298e-01, -7.7224e-01, -4.7089e-01, -1.0953e+00, -1.6847e+00,
         -1.9500e+00, -1.5561e+00, -2.

In [56]:
out.max()

tensor(16.6003, grad_fn=<MaxBackward1>)

In [61]:
with open('../../dlwpt-code-master/data/p1ch2/imagenet_classes.txt') as f:
    labels = [line.strip() for line in f.readlines()]

In [62]:
_, index = torch.max(out, 1)  # 取出最大值的索引

In [63]:
index

tensor([207])

In [66]:
percentage = torch.nn.functional.softmax(out, dim=1)[0] * 100 # softmax: normalize our outputs to the range [0, 1]
labels[index[0]], percentage[index[0]].item()  # index是一个单元素的tensor，所以需要通过索引取出。 item()也是取出元素的值，但精度更高

('golden retriever', 97.63997650146484)

In [68]:
_, indices = torch.sort(out, descending=True) # 降序排列
[(labels[idx], percentage[idx].item()) for idx in indices[0][:5]]

[('golden retriever', 97.63997650146484),
 ('Labrador retriever', 1.6723159551620483),
 ('cocker spaniel, English cocker spaniel, cocker', 0.41784098744392395),
 ('tennis ball', 0.08509056270122528),
 ('redbone', 0.03111395426094532)]